## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
from config import weather_api_key

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain (inches),Snow (inches)
0,0,Nizwa,OM,22.9333,57.5333,90.03,40,14,2.64,few clouds,0.00,0
1,1,Hermanus,ZA,-34.4187,19.2345,55.90,80,22,6.96,few clouds,0.00,0
2,2,Bambous Virieux,MU,-20.3428,57.7575,68.25,68,40,12.66,light rain,0.13,0
3,3,Cape Town,ZA,-33.9258,18.4232,56.12,87,0,2.30,clear sky,0.00,0
4,4,Mar Del Plata,AR,-38.0023,-57.5575,53.62,87,0,10.36,clear sky,0.00,0


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))
rain_bool = input("Do you want it to be raining? (yes/no) ")
snow_bool = input("Do you want it to be snowing? (yes/no) ")

What is the minimum temperature you would like for your trip? 53.62
What is the maximum temperature you would like for your trip? 90.03
Do you want it to be raining? (yes/no) yes
Do you want it to be snowing? (yes/no) yes


In [13]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
if rain_bool == "yes" and snow_bool == "yes":
    filtered_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp) & \
                                       (city_data_df["Rain (inches)"] == 0) & \
                                        (city_data_df["Snow (inches)"] == 0)]
elif rain_bool == "no" and snow_bool == "yes":
    filtered_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp) & \
                                       (city_data_df["Rain (inches)"] == 0) & \
                                        (city_data_df["Snow (inches)"] >= 0.0)]
elif rain_bool == "yes" and snow_bool == "no":
    filtered_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp) & \
                                       (city_data_df["Rain (inches)"] >= 0.0) & \
                                        (city_data_df["Snow (inches)"] == 0.0)]
else:
    filtered_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp) & \
                                       (city_data_df["Rain (inches)"] >= 0.0) & \
                                        (city_data_df["Snow (inches)"] >= 0.0)]

In [14]:
# 4a. Determine if there are any empty rows.
clean_df = filtered_cities_df.dropna()

In [15]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain (inches),Snow (inches)
0,0,Nizwa,OM,22.9333,57.5333,90.03,40,14,2.64,few clouds,0.0,0
1,1,Hermanus,ZA,-34.4187,19.2345,55.90,80,22,6.96,few clouds,0.0,0
3,3,Cape Town,ZA,-33.9258,18.4232,56.12,87,0,2.30,clear sky,0.0,0
4,4,Mar Del Plata,AR,-38.0023,-57.5575,53.62,87,0,10.36,clear sky,0.0,0
5,5,Asosa,ET,10.0667,34.5333,59.99,95,100,1.32,overcast clouds,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
571,571,Mitsamiouli,KM,-11.3847,43.2844,77.04,71,7,5.82,clear sky,0.0,0
573,573,Shache,CN,38.4167,77.2406,72.23,27,100,1.81,overcast clouds,0.0,0
574,574,Bratsk,RU,56.1325,101.6142,55.89,82,100,4.18,overcast clouds,0.0,0
576,576,Ucluelet,CA,48.9329,-125.5528,83.05,80,45,7.27,scattered clouds,0.0,0


In [16]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Nizwa,OM,90.03,few clouds,22.9333,57.5333,
1,Hermanus,ZA,55.90,few clouds,-34.4187,19.2345,
3,Cape Town,ZA,56.12,clear sky,-33.9258,18.4232,
4,Mar Del Plata,AR,53.62,clear sky,-38.0023,-57.5575,
5,Asosa,ET,59.99,overcast clouds,10.0667,34.5333,
6,Castro,BR,62.11,scattered clouds,-24.7911,-50.0119,
7,Edson,CA,75.34,few clouds,53.5835,-116.4356,
10,Rognan,NO,53.89,overcast clouds,67.0953,15.3878,
12,Palmer,US,83.03,scattered clouds,42.1584,-72.3287,
14,Nikolskoye,RU,60.03,scattered clouds,59.7035,30.7861,


In [17]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"] 
    lng = row["Lng"]
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    params["location"] = f"{lat},{lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:    
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not foun

In [18]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df = hotel_df.dropna()

In [22]:
# 8a. Create the output File (CSV)
output_data_file = "../Weather_Database/WeatherPy_Vacation.csv"

# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label = "City_ID")

In [23]:
gmaps.configure(api_key= g_key)

In [24]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>

"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [25]:
# 11a. Add a marker layer for each city to the map.
locations = hotel_df[["Lat", "Lng"]]
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
fig.add_layer(marker_layer)

In [26]:
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))